# EDA Image Aquisation for Grid-Model
In this notebook we train a model with satellite images to classify the probability of an traffic accident 
Therefore we do the following steps:
- Choose City: We have to choose a city that we want to train our model with 
- Raster über Stadt legen 
- Für jedes Raster Anzahl der Unfälle herausfinden 
- Anzahl der Kategorien + Zuordnoungskriterien für Kategorien erarbeiten 
- Raster Kategorien zuordnen



## Choose City

In [3]:
import pandas as pd
import numpy as np
from rtree import index
import geopandas as gpd
from shapely.geometry import Point


In [4]:
data = pd.read_csv('C:/Projekte/TDS/TDS2324-TrafficAccidents/Data/TrafficAccidentData/all_16_22.csv',dtype={'ags': str})
data.head()

,Unnamed: 0,land,regbez,kreis,gemeinde,jahr,monat,stunde,wochentag,kategorie,...,ist_pkw,ist_fuss,ist_krad,ist_gkfz,ist_sonstige,linrefx,linrefy,xgcswgs84,ygcswgs84,ags
0,0,1,0,53,120,2016,1,9,5,2,...,1,0,0,0.0,0,606982.3940,5.954660e+06,10.621659,53.729615,01053120
1,1,1,0,57,10,2016,1,17,3,3,...,1,0,0,0.0,0,574882.5330,6.011441e+06,10.149176,54.245453,01057010
2,2,1,0,62,8,2016,1,0,5,3,...,1,0,0,0.0,0,599934.6875,5.964609e+06,10.518094,53.820403,01062008
3,3,1,0,3,0,2016,1,15,5,3,...,0,0,0,0.0,1,610709.3487,5.968284e+06,10.683021,53.851243,01003000
4,4,1,0,55,28,2016,1,14,1,3,...,1,0,0,0.0,0,605690.7904,6.009152e+06,10.620986,54.219459,01055028


In [5]:
# check wich german city has most accidents
most_accidents_ags = data['ags'].value_counts()
most_accidents_ags.head()


ags
09162000    33806
06412000    17220
05315000    16671
04011000    15672
14612000    14513
Name: count, dtype: int64

The AGS stands for:
1. Munich
2. Frankfurt
3. Cologne
4. Bremen
5. Dresden

Since we have very good satellite images and a lot of accident data for Munich, we will train our model with the city of Munich

## Data Introduction Munich

In [6]:
# saving the data for the city of munich
munich = data[data['ags'] == '09162000']
munich.head()

,Unnamed: 0,land,regbez,kreis,gemeinde,jahr,monat,stunde,wochentag,kategorie,...,ist_pkw,ist_fuss,ist_krad,ist_gkfz,ist_sonstige,linrefx,linrefy,xgcswgs84,ygcswgs84,ags
60594,60599,9,1,62,0,2016,1,16,1,3,...,1,0,0,0.0,0,694645.8088,5.332073e+06,11.615121,48.112151,09162000
60680,60685,9,1,62,0,2016,1,14,1,3,...,1,0,0,0.0,0,687820.0791,5.333578e+06,11.524172,48.127725,09162000
60853,60858,9,1,62,0,2016,1,15,1,2,...,0,0,0,0.0,0,688689.1970,5.335742e+06,11.536799,48.146921,09162000
60953,60958,9,1,62,0,2016,1,3,6,3,...,1,1,0,0.0,0,700587.1593,5.332651e+06,11.695126,48.115499,09162000
61071,61076,9,1,62,0,2016,1,14,3,3,...,1,0,0,0.0,0,694222.7100,5.340374e+06,11.613230,48.186879,09162000


In [7]:
munich.shape

(33806, 25)

## Building the Grid
- to get a rectangular form of munich we are looking for the most min and maximum longitude and latitude so we can build our grid.


In [8]:
# Finding the boundaries of the grid 
min_longitude = munich['xgcswgs84'].min()
max_longitude = munich['xgcswgs84'].max()
min_latitude = munich['ygcswgs84'].min()
max_latitude = munich['ygcswgs84'].max()


- Afterwards we estimate the degree of coordinates in meters, to build our grid in a 30m x 30m grid.

In [9]:
# Constants for conversion
METERS_PER_DEGREE_LATITUDE = 111000  # approximately 111 kilometers
METERS_PER_DEGREE_LONGITUDE = 71000  # approximately 71 kilometers at Munich's latitude

# Calculating degree increments for a 30 meter cell
degree_increment_latitude = 30 / METERS_PER_DEGREE_LATITUDE
degree_increment_longitude = 30 / METERS_PER_DEGREE_LONGITUDE

Now we can generate the grid

In [10]:
# Generating the grid
latitude_range = np.arange(min_latitude, max_latitude, degree_increment_latitude)
longitude_range = np.arange(min_longitude, max_longitude, degree_increment_longitude)

# Creating a DataFrame to represent the grid
grid_cells = pd.DataFrame([(lat, lon) for lat in latitude_range for lon in longitude_range], 
                          columns=['Latitude', 'Longitude'])

grid_cells.head()

,Latitude,Longitude
0,48.065872,11.381973
1,48.065872,11.382395
2,48.065872,11.382818
3,48.065872,11.383240
4,48.065872,11.383663


The coordinates show the upper left corner of a grid cell. 

In [11]:
grid_cells.shape

(516928, 2)

we have around 516928 30m x 30m grid cells. We export them as CSV and check if the grid cells are as expected using QGIS. The QGIS project can be found Data\QGIS\Munich grid.

In [12]:
# export munich grid as csv
grid_cells.to_csv('C:/Projekte/TDS/TDS2324-TrafficAccidents/Data/Munich/munich_grid.csv', index=False)

## Categorize grid cells

Now we going to look deeper into the grid cells and analyse the dristribution of accdients in the grid cells to create some categories in which we can sort them. to make the runtime for the code faster we use a R-Tree. 

In [13]:
from rtree import index

# Create an R-tree index
idx = index.Index()

# Populate the index with the bounding boxes of the grid cells
for i, row in grid_cells.iterrows():
    # Assume that 'Longitude' and 'Latitude' are the lower bounds of the cell
    # and 'Longitude_max' and 'Latitude_max' are the upper bounds
    idx.insert(i, (row['Longitude'], row['Latitude'], row['Longitude']+degree_increment_longitude, row['Latitude']+degree_increment_latitude))

# Add a 'grid_cell' column to the munich DataFrame
munich['grid_cell'] = np.nan

# Assign a grid cell to each accident
for i, row in munich.iterrows():
    # Find the first grid cell that contains the accident
    for j in idx.intersection((row['xgcswgs84'], row['ygcswgs84'])):
        munich.at[i, 'grid_cell'] = j
        break

munich.head()

C:\Users\miche\AppData\Local\Temp\ipykernel_4308\3438720557.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  munich['grid_cell'] = np.nan


,Unnamed: 0,land,regbez,kreis,gemeinde,jahr,monat,stunde,wochentag,kategorie,...,ist_fuss,ist_krad,ist_gkfz,ist_sonstige,linrefx,linrefy,xgcswgs84,ygcswgs84,ags,grid_cell
60594,60599,9,1,62,0,2016,1,16,1,3,...,0,0,0.0,0,694645.8088,5.332073e+06,11.615121,48.112151,09162000,135299.0
60680,60685,9,1,62,0,2016,1,14,1,3,...,0,0,0.0,0,687820.0791,5.333578e+06,11.524172,48.127725,09162000,180000.0
60853,60858,9,1,62,0,2016,1,15,1,2,...,0,0,0.0,0,688689.1970,5.335742e+06,11.536799,48.146921,09162000,235978.0
60953,60958,9,1,62,0,2016,1,3,6,3,...,1,0,0.0,0,700587.1593,5.332651e+06,11.695126,48.115499,09162000,144945.0
61071,61076,9,1,62,0,2016,1,14,3,3,...,0,0,0.0,0,694222.7100,5.340374e+06,11.613230,48.186879,09162000,352783.0


In [14]:
# save munich with grid cell as csv
munich.to_csv('C:/Projekte/TDS/TDS2324-TrafficAccidents/Data/Munich/munich.csv', index=False)

Now we find out how many accidents happen in each gridcell

In [15]:
# add grid_cells a feature named count
grid_cells['count'] = 0

# Count the number of accidents in each grid cell
for i, row in munich.iterrows():
    grid_cells.at[row['grid_cell'], 'count'] += 1

grid_cells.head()

,Latitude,Longitude,count
0,48.065872,11.381973,0
1,48.065872,11.382395,0
2,48.065872,11.382818,0
3,48.065872,11.383240,0
4,48.065872,11.383663,0


Now we find out how many cells have a certain accident_count

In [16]:
# make new empty df
count_cells = pd.DataFrame(columns=['accident_count', 'amount_cells'])

# fill accident_count with uniquevalues from count from grid_cells
count_cells['accident_count'] = grid_cells['count'].unique()

# fill amount_cells with the amount of cells with the same accident_count
for i, row in count_cells.iterrows():
    count_cells.at[i, 'amount_cells'] = grid_cells[grid_cells['count'] == row['accident_count']].shape[0]

count_cells.sort_values(by=['accident_count'], inplace=True)
count_cells.head()





,accident_count,amount_cells
0,0,501493
1,1,9148
2,2,2852
4,3,1219
5,4,698


Since the Distribution is very diverse we try to build our categories in a way we that we have around 1000 Pictures foreach category 

In [17]:
# categorize each grid cell by the number of accidents it contains
#category 1 = 0 accidents
#category 2 = 1 accidents
#category 3 = 2 accidents
#category 4 = 3 accidents
#category 5 = 4-5 accidents
#category 6 = 6+ accidents
grid_cells['category'] = 0

for i, row in grid_cells.iterrows():
    if row['count'] == 0:
        grid_cells.at[i, 'category'] = 1
    elif row['count'] == 1:
        grid_cells.at[i, 'category'] = 2
    elif row['count'] == 2:
        grid_cells.at[i, 'category'] = 3
    elif row['count'] == 3:
        grid_cells.at[i, 'category'] = 4
    elif row['count'] == 4 or row['count'] == 5:
        grid_cells.at[i, 'category'] = 5
    else:
        grid_cells.at[i, 'category'] = 6

grid_cells.head()


,Latitude,Longitude,count,category
0,48.065872,11.381973,0,1
1,48.065872,11.382395,0,1
2,48.065872,11.382818,0,1
3,48.065872,11.383240,0,1
4,48.065872,11.383663,0,1


In [18]:
# how often does each category occur?
grid_cells['category'].value_counts()


category
1    501493
2      9148
3      2852
4      1219
5      1140
6      1076
Name: count, dtype: int64

In [19]:
# Filter the munich DataFrame to include only the WGS coordinates
munich_wgs = munich[['xgcswgs84', 'ygcswgs84']]

# Save the filtered DataFrame as a CSV file
munich_wgs.to_csv('C:/Projekte/TDS/TDS2324-TrafficAccidents/Data/Munich/munich_coord.csv', index=False)



## Selecting cells
Here we select the cells we download the pictures from.

In [20]:
# Create a DataFrame with 1000 cells of each category
category_1 = grid_cells[grid_cells['category'] == 1].sample(n=1000, replace=True)
category_2 = grid_cells[grid_cells['category'] == 2].sample(n=1000, replace=True)
category_3 = grid_cells[grid_cells['category'] == 3].sample(n=1000, replace=True)
category_4 = grid_cells[grid_cells['category'] == 4].sample(n=1000, replace=True)
category_5 = grid_cells[grid_cells['category'] == 5].sample(n=1000, replace=True)
category_6 = grid_cells[grid_cells['category'] == 6].sample(n=1000, replace=True)

# Concatenate the DataFrames
selected_img = pd.concat([category_1, category_2, category_3, category_4, category_5, category_6])

# Reset the index of the DataFrame
selected_img.reset_index(drop=True, inplace=True)

# Print the resulting DataFrame
print(selected_img)

# print how often each category occurs
selected_img['category'].value_counts()

# save selected_img as csv
selected_img.to_csv('C:/Projekte/TDS/TDS2324-TrafficAccidents/Data/Images/selected_img_grid.csv', index=False)



       Latitude  Longitude  count  category
0     48.108574  11.416198      0         1
1     48.199385  11.541269      0         1
2     48.183169  11.523522      0         1
3     48.232628  11.417466      0         1
4     48.135601  11.444508      0         1
...         ...        ...    ...       ...
5995  48.145061  11.578874     17         6
5996  48.106682  11.646057      6         6
5997  48.148845  11.562818      9         6
5998  48.156682  11.558170      6         6
5999  48.129385  11.558170     22         6

[6000 rows x 4 columns]
